# Step 7： Data Mining

# First Model: Random forest

In [1]:
# Must be included at the beginning of each new notebook. Remember to change the app name.
import findspark
findspark.init('/home/ubuntu/spark-3.2.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('DBAS-Step4-Data Transformation').getOrCreate()

# Enable pandas-on-Spark
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/10 09:42:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
## Load data from csv file
spk_df = spark.read.csv("Data/4DT/heart_failure_dataset_4DT.csv", header=True, inferSchema=True)

# Convert Spark DataFrame to pandas-on-Spark DataFrame using to_pandas_on_spark()
spkpd_df = spk_df.to_pandas_on_spark()
spkpd_df.info()


/home/ubuntu/.local/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
23/10/10 09:42:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/10 09:42:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


<class 'pyspark.pandas.frame.DataFrame'>
Int64Index: 383 entries, 0 to 382
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   creatinine_phosphokinase  383 non-null    int32  
 1   ejection_fraction         383 non-null    int32  
 2   platelets                 383 non-null    float64
 3   serum_creatinine          383 non-null    float64
 4   serum_sodium              383 non-null    int32  
 5   DEATH_EVENT               383 non-null    int32  
dtypes: float64(2), int32(4)

In [4]:
from pyspark.ml.feature import VectorAssembler, StandardScaler
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim


ModuleNotFoundError: No module named 'torch'

In [ ]:
# 06-DMA

# Load relevant algorithms for Nueral Network model
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.linalg import DenseVector
from sklearn.model_selection import train_test_split
#from tensorflow import keras
from sklearn.metrics import accuracy_score


In [ ]:
### ---------------- 07-DM-NN

# Prepare data for MLlib
feature_cols = [col for col in spk_df.columns if col != 'DEATH_EVENT']
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df_assembled = assembler.transform(spk_df)

# Feature scaling
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
scaler_model = scaler.fit(df_assembled)
df_scaled = scaler_model.transform(df_assembled)

# Convert to pandas for train-test split and neural network training
pdf = df_scaled.select("scaledFeatures", "DEATH_EVENT").toPandas()
X = pdf["scaledFeatures"].apply(lambda x: x.toArray()).tolist()
y = pdf["DEATH_EVENT"].tolist()

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create neural network model
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(len(X_train[0]),)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train using neural network  
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=0)

# Predict and evaluate accuracy
y_pred = (model.predict(X_test) > 0.5).astype("int32")
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy of Neural Network model: {accuracy*100:.2f}%")


In [ ]:
### ---------------- 07-DM-NN

X = df.drop("DEATH_EVENT", axis=1)
y = df["DEATH_EVENT"]

# Feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Create neural network model
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# Train using neural network  
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=0)

# Predict and evaluate accuracy
y_pred = (model.predict(X_test) > 0.5).astype("int32")
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy of Nueral Network model: {accuracy*100:.2f}%")

In [ ]:
# ---- Use SHAP to assess feature importance
explainer = shap.DeepExplainer(model, X_train)
shap_values = explainer.shap_values(X_test)

# Calculate the average absolute SHAP value for each feature
mean_shap_values = np.abs(shap_values[0]).mean(axis=0)

# Create a DataFrame to store the result
shap_df = pd.DataFrame({
    'feature': X.columns,
    'importance': mean_shap_values
})

# Sort features based on importance
shap_df = shap_df.sort_values(by='importance', ascending=True)

# Plotting feature importance
plt.figure(figsize=(10, 6))
plt.barh(shap_df['feature'], shap_df['importance'], color='skyblue')
plt.xlabel('Mean Absolute SHAP Value')
plt.title('Feature Importance using SHAP')
plt.show()

In [ ]:
# Stop Spark session
spark.stop()